In [3]:
import numpy as np
import pandas as pd
from itertools import combinations
import math, statistics 

In [101]:
np.set_printoptions(precision=2)

## 9.3 Collaborative Filtering

### 9.3.1: The below matrix represents the ratings, on a 1-5 star scale, of eight items, a through h, by three users A, B, and C. Compute the following from the data of this matrix.

In [5]:
utilityMatrix = np.array([[4, 5, None, 5, 1, None, 3, 2], 
                           [None, 3, 4, 3, 1, 2, 1, None],
                           [2, None, 1, 3, None, 4, 5, 3]], dtype=np.float)
items = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
users = ['A', 'B', 'C']

In [6]:
utilityFrameO = pd.DataFrame(utilityMatrix, columns=items, index=users)
utilityFrame = utilityFrameO.copy()
utilityFrame

,a,b,c,d,e,f,g,h
A,4.0,5.0,NaN,5.0,1.0,NaN,3.0,2.0
B,NaN,3.0,4.0,3.0,1.0,2.0,1.0,NaN
C,2.0,NaN,1.0,3.0,NaN,4.0,5.0,3.0


#### (a) Treating the utility matrix as a boolean, compute the Jaccard distance between each pair of users.

In [7]:
binaryFrame = pd.DataFrame((np.isnan(utilityMatrix)).astype(int), columns=items, index=users)
binaryFrame

,a,b,c,d,e,f,g,h
A,0,0,1,0,0,1,0,0
B,1,0,0,0,0,0,0,1
C,0,1,0,0,1,0,0,0


In [8]:
def JaccardDistance(v1, v2):
    return 1 - (np.sum(np.abs(v1 - v2)) / len(v1))

In [9]:
for userPair in combinations(users, 2):
    print(userPair[0] + ',' + userPair[1] + ' Jaccard distance: ' + str(JaccardDistance(binaryFrame.loc[userPair[0]], binaryFrame.loc[userPair[1]])))

A,B Jaccard distance: 0.5
A,C Jaccard distance: 0.5
B,C Jaccard distance: 0.5


#### (b) Repeat Part (a), but use the cosine distance.

In [10]:
def CosineDistance(v1, v2):
    dotProduct = np.dot(v1, v2)
    v1Norm = np.linalg.norm(v1)
    v2Norm = np.linalg.norm(v2)
    return dotProduct / (v1Norm * v2Norm)

In [11]:
for userPair in combinations(users, 2):
    a = utilityFrame.loc[userPair[0]]
    b = utilityFrame.loc[userPair[1]]
    a[np.isnan(a)] = 0
    b[np.isnan(b)] = 0
    print(userPair[0] + ',' + userPair[1] + ' Cosine Distance: ' + str(CosineDistance(a,b)))

A,B Cosine Distance: 0.6010407640085653
A,C Cosine Distance: 0.6149186938124421
B,C Cosine Distance: 0.5138701197773616


#### (c) Treat ratings of 3, 4, and 5 as 1 and 1, 2, and blank as 0. Compute the following from the data of this matrix.

In [12]:
alteredFrame = utilityFrame.copy()
alteredFrame[alteredFrame <= 2] = 0
alteredFrame[alteredFrame > 2] = 1
alteredFrame

,a,b,c,d,e,f,g,h
A,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
B,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


In [13]:
for userPair in combinations(users, 2):
    print(userPair[0] + ',' + userPair[1] + ' Jaccard distance: ' + str(JaccardDistance(alteredFrame.loc[userPair[0]], alteredFrame.loc[userPair[1]])))

A,B Jaccard distance: 0.625
A,C Jaccard distance: 0.5
B,C Jaccard distance: 0.375


#### (d) Repeat Part (c), but use the consine distance.

In [14]:
for userPair in combinations(users, 2):
    a = alteredFrame.loc[userPair[0]]
    b = alteredFrame.loc[userPair[1]]
    a[np.isnan(a)] = 0
    b[np.isnan(b)] = 0
    print(userPair[0] + ',' + userPair[1] + ' Cosine distance: ' + str(JaccardDistance(a, b)))

A,B Cosine distance: 0.625
A,C Cosine distance: 0.5
B,C Cosine distance: 0.375


#### (e) Normalize the matrix by subtracting from each nonblank entry the average value of its user.

In [15]:
normalisedMatrix = np.apply_along_axis(lambda x: x - statistics.mean(x[[not bool for bool in np.isnan(x)]]), 1, utilityFrameO)
normalisedFrame = pd.DataFrame(normalisedMatrix, columns=items, index=users)
normalisedFrame

,a,b,c,d,e,f,g,h
A,0.666667,1.666667,NaN,1.666667,-2.333333,NaN,-0.333333,-1.333333
B,NaN,0.666667,1.666667,0.666667,-1.333333,-0.333333,-1.333333,NaN
C,-1.000000,NaN,-2.000000,0.000000,NaN,1.000000,2.000000,0.000000


#### (f) Using the normalized matrix from Part (e), compute the cosine distance between each pair of users.

In [16]:
for userPair in combinations(users, 2):
    a = normalisedFrame.loc[userPair[0]]
    b = normalisedFrame.loc[userPair[1]]
    a[np.isnan(a)] = 0
    b[np.isnan(b)] = 0
    print(userPair[0] + ',' + userPair[1] + ' Cosine distance: ' + str(JaccardDistance(a, b)))

A,B Cosine distance: 0.0
A,C Cosine distance: -0.75
B,C Cosine distance: -0.5


### 9.3.2: In this exercise, we cluster items in the utility matrix.

#### (a) Cluster the right items hierarchically into four clusters. The following method should be used to cluster. Replace all 3's, 4's and 5's by 1 and replace 1's, 2's, and blanks by 0. Use the Jaccard distance to measure the distance between clusters of more than one element, take the distance between clusters to be the minimum distance between pairs of elements, one from each cluster.

In [17]:
numOfClusters = 4
clusterFrame = alteredFrame.copy()
clusterItems = items.copy()
clusterFrame

,a,b,c,d,e,f,g,h
A,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
B,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


In [18]:
def CalculateProximityMatrix(proxItems, proxFrame):
    proximityMatrix = np.empty((len(proxItems), len(proxItems)), dtype = np.float)
    for itemInd1 in range(len(proxItems)):
        for itemInd2 in range(itemInd1, len(proxItems)):
            distance = JaccardDistance(proxFrame.loc[:,proxItems[itemInd1]], proxFrame.loc[:,proxItems[itemInd2]])
            if itemInd1 == itemInd2:
                distance = None
            proximityMatrix[itemInd1, itemInd2] = distance
            proximityMatrix[itemInd2, itemInd1] = distance
    return 1 - proximityMatrix

In [19]:
while len(clusterFrame.columns) > numOfClusters:
    proximityMatrix = CalculateProximityMatrix(clusterItems, clusterFrame)
    minIndexes = np.unravel_index(np.nanargmin(proximityMatrix), proximityMatrix.shape)
    item1 = clusterItems[minIndexes[0]]
    item2 = clusterItems[minIndexes[1]]
    clusterFrame[item1+item2] = pd.concat([clusterFrame.loc[:,item1], clusterFrame.loc[:,item2]], axis=1).mean(axis=1)
    clusterFrame = clusterFrame.drop(item1, 1)
    clusterFrame = clusterFrame.drop(item2, 1)
    clusterItems.append(item1+item2)
    clusterItems.remove(item1)
    clusterItems.remove(item2)

#### (b) Then, construct from the original matrix a new matrix whose rows correspond to users, as before, and whose columns correspond to clusters. Compute the entry of a user and cluster of items by averaging the nonblank entries for that user and all the items in the cluster.

In [20]:
clusterFrame

,fh,ab,ce,dg
A,0.0,1.0,0.0,1.0
B,0.0,0.5,0.5,0.5
C,1.0,0.0,0.0,1.0


#### (d) Compute the cosine distance between each pair of users, according to the matrix from Part (b)

In [21]:
for userPair in combinations(users, 2):
    a = clusterFrame.loc[userPair[0]]
    b = clusterFrame.loc[userPair[1]]
    a[np.isnan(a)] = 0
    b[np.isnan(b)] = 0
    print(userPair[0] + ',' + userPair[1] + ' Cosine distance: ' + str(JaccardDistance(a, b)))

A,B Cosine distance: 0.625
A,C Cosine distance: 0.5
B,C Cosine distance: 0.375


## 9.4 Dimensionality Reduction

### 9.3.1: Starting with the decomposition of the matrix below, we may choose any of the 20 entries in U or V to optimise first. Perform this first optimisation step.

In [63]:
U = np.ones(shape = (5, 2), dtype = np.float)
V = np.ones(shape = (2, 5), dtype = np.float)
M = np.array([[5, 2, 4, 4, 3],
              [3, 1, 2, 4, 1],
              [2, None, 3, 1, 4],
              [2, 5, 4, 3, 5],
              [4, 4, 5, 4, None]], dtype = np.float)
M

array([[ 5.,  2.,  4.,  4.,  3.],
       [ 3.,  1.,  2.,  4.,  1.],
       [ 2., nan,  3.,  1.,  4.],
       [ 2.,  5.,  4.,  3.,  5.],
       [ 4.,  4.,  5.,  4., nan]])

#### (a) on u32

In [64]:
def SolveForX(r, s):
    Vs2 = np.sum(np.square(V[s,]))
    diff = np.nansum(np.multiply(V[s,], (M[r,] - (U[r,-s] * V[-s,]))))
    return diff/Vs2

In [65]:
SolveForX(2, 1)

1.2

#### (b) on v41

In [66]:
def SolveForY(r, s):
    Ur2 = np.sum(np.square(U[:,r]))
    diff = np.nansum(np.multiply(U[:,r], (M[:,s] - (U[:,-r] * V[-r,s]))))
    return diff/Ur2

In [68]:
SolveForY(0, 3)

2.2

### 9.4.2 If we wish to start out with all U and V entries set to the same value, what value minimizes the RMSE for the matrix M?

In [69]:
def RMSE(trueM, testM):
    return np.sqrt(np.nanmean(np.square(trueM - testM)))

In [70]:
def SecondDerivative(x):
    d2 = np.nansum(M) - (4 * x)
    if d2 > 0:
        print('Min')
    if d2 < 0:
        print('Max')

Original RMSE

In [71]:
RMSE(M, np.matmul(U,V))

1.805787796286538

Using 2x<sup>2</sup> as P<sub>ij</sub> and solving for x, we get the following:

In [72]:
x = np.sqrt((0.5*np.nansum(M)) / (M.shape[0]*M.shape[1]))
x

1.224744871391589

In [73]:
SecondDerivative(x)

Min


In [74]:
UV = np.full((5,5), 2 * math.pow(x,2), dtype = np.float)
UV

array([[3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3.]])

In [75]:
RMSE(M, UV)

1.318760946791574

### 9.4.3: Starting with U and V as below

In [76]:
U = np.array([[2.6, 1, 1.76, 1, 1],
              [1, 1, 1, 1, 1]], dtype = np.float).T
V = np.array([[1.617, 1, 1, 1, 1],
              [1, 1, 1, 1, 1]], dtype = np.float)

#### (a) Reconsider the value of u<sub>11</sub>. Find its new best value given the changes that have been made so far.

In [79]:
newX = SolveForX(0,0)
newX

0.5876026219826811

In [80]:
U[0,0] = newX

#### (b) Then choose the best value for u<sub>52</sub>

In [81]:
newX = SolveForX(4,1)
newX

2.6

In [82]:
U[4,1] = newX

#### (c) Then choose the best value for v<sub>22</sub>

In [83]:
newY = SolveForY(1,1)
newY

0.8029739776951672

In [84]:
V[1,1] = newY

### 9.4.5: Normalise the matrix M of our running example by:

In [85]:
M = np.matmul(U,V)
M

array([[1.95, 1.39, 1.59, 1.59, 1.59],
       [2.62, 1.8 , 2.  , 2.  , 2.  ],
       [3.85, 2.56, 2.76, 2.76, 2.76],
       [2.62, 1.8 , 2.  , 2.  , 2.  ],
       [4.22, 3.09, 3.6 , 3.6 , 3.6 ]])

#### (a) First subtracting from each element the average of its row, and then subtracting from each element the average of its (modified) column.

In [88]:
normMa = np.apply_along_axis(lambda x: x - statistics.mean(x), 1, M)
normMa = np.apply_along_axis(lambda x: x - statistics.mean(x), 0, normMa)
normMa

array([[-0.25,  0.11,  0.05,  0.05,  0.05],
       [-0.05,  0.06, -0.  , -0.  , -0.  ],
       [ 0.33, -0.03, -0.1 , -0.1 , -0.1 ],
       [-0.05,  0.06, -0.  , -0.  , -0.  ],
       [ 0.02, -0.19,  0.06,  0.06,  0.06]])

#### (a) First subtracting from each element the average of its column, and then subtracting from each element the average of its (modified) row.

In [89]:
normMb = np.apply_along_axis(lambda x: x - statistics.mean(x), 0, M)
normMb = np.apply_along_axis(lambda x: x - statistics.mean(x), 1, normMb)
normMb

array([[-0.25,  0.11,  0.05,  0.05,  0.05],
       [-0.05,  0.06, -0.  , -0.  , -0.  ],
       [ 0.33, -0.03, -0.1 , -0.1 , -0.1 ],
       [-0.05,  0.06, -0.  , -0.  , -0.  ],
       [ 0.02, -0.19,  0.06,  0.06,  0.06]])

Are there any differences in the results of (a) and (b)?

In [96]:
np.allclose(normMa,normMb)

True

In [97]:
np.array_equal(normMa, normMb)

False

In [102]:
normMa - normMb

array([[-1.11e-16, -1.67e-16, -8.33e-17, -8.33e-17, -8.33e-17],
       [-5.55e-17, -1.11e-16, -2.78e-17, -2.78e-17, -2.78e-17],
       [ 1.11e-16,  5.55e-17,  1.39e-16,  1.39e-16,  1.39e-16],
       [-5.55e-17, -1.11e-16, -2.78e-17, -2.78e-17, -2.78e-17],
       [ 1.11e-16,  5.55e-17,  1.39e-16,  1.39e-16,  1.39e-16]])

Close enough :)